In [1]:
!pip install requests openpyxl pandas


In [ ]:
import requests
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

def get_github_repository_vulnerabilities(github_token, repository):
    headers = {
        'Authorization': f'Bearer {github_token}'
    }

    response = requests.get(f'https://api.github.com/repos/{repository}/vulnerability-alerts', headers=headers)

    if response.status_code == 200:
        return response.json().get('data', {}).get('securityVulnerabilities', [])
    else:
        print(f"Error getting vulnerabilities from GitHub API: {response.status_code}")
        return []

def get_snyk_project_vulnerabilities(snyk_token, org, project):
    headers = {
        'Authorization': f'token {snyk_token}'
    }

    response = requests.get(f'https://snyk.io/api/v1/org/{org}/project/{project}', headers=headers)

    if response.status_code == 200:
        return response.json().get('vulnerabilities', [])
    else:
        print(f"Error getting vulnerabilities from Snyk API: {response.status_code}")
        return []

def main():
    github_token = 'YOUR_GITHUB_TOKEN'
    snyk_token = 'YOUR_SNKY_TOKEN'
    github_repository = 'owner/repo'  # replace with your GitHub repository in the 'owner/repo' format
    snyk_organization = 'your-snyk-org'
    snyk_project = 'your-snyk-project'

    github_vulnerabilities = get_github_repository_vulnerabilities(github_token, github_repository)
    snyk_vulnerabilities = get_snyk_project_vulnerabilities(snyk_token, snyk_organization, snyk_project)

    # Process and combine data as needed
    combined_data = {
        'GitHub Vulnerabilities': github_vulnerabilities,
        'Snyk Vulnerabilities': snyk_vulnerabilities,
    }

    # Export data to Excel
    wb = Workbook()
    for sheet_name, data in combined_data.items():
        df = pd.DataFrame(data)
        ws = wb.create_sheet(title=sheet_name)
        for row in dataframe_to_rows(df, index=False, header=True):
            ws.append(row)

    # Save the Excel file
    wb.save('vulnerability_data.xlsx')

if __name__ == "__main__":
    main()
